In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import math

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dropout, BatchNormalization, LeakyReLU, Activation
from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import keras
from keras.models import Sequential, Model, model_from_json
from keras.layers import Dense, Conv2D, Activation, MaxPool2D, Flatten, Dropout, BatchNormalization
from keras.utils import np_utils
from keras.preprocessing import image
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam

In [ ]:
print('CPU: {}'.format(len(tf.config.list_physical_devices('CPU'))))
print('GPU: {}'.format(len(tf.config.list_physical_devices('GPU'))))

In [ ]:
data = pd.read_csv('/kaggle/input/facial-expression-recognitionferchallenge/fer2013/fer2013/fer2013.csv')
data.shape

**Veri setine ilk bakış**.

In [ ]:
data.head()

In [ ]:
data["Usage"].value_counts()

Veri Ön İşleme Kısmı

In [ ]:
trainData = data[data.Usage == "Training"] #Sadece eğitim verilerini trainData olarak tanımladık.

In [ ]:
trainLabelsFlat = trainData["emotion"].values.ravel()
trainLabelsCount = np.unique(trainLabelsFlat).shape[0]

In [ ]:
#Train verileri dataframe'in içinde bize tablo olarak verilmiştir. Boşluklara göre bölerek her bir footoğrafa ait pixel değerini listeye atıyorum.
trainPixel = trainData.pixels.str.split(" ").tolist()

trainPixel = pd.DataFrame(data = trainPixel, dtype = int)
trainImages = trainPixel.values
trainImages = trainImages.astype(np.float)

In [ ]:
def show(img):
    showImg = img.reshape(48,48)
    
    plt.axis("off")
    plt.imshow(showImg, cmap = 'gray')

In [ ]:
#Fonksiyonumuzu deneyelim
show(trainImages[45])

In [ ]:
#Verilerin her birine düşen sınıfı yani eğitim işlemi boyutunu alıyoruz.
def dense_to_one_hot(labelsDense, numClasses):
    numLabels = labelsDense.shape[0]
    indexOffset = np.arange(numLabels) * numClasses
    labelsOneHot = np.zeros((numLabels, numClasses))
    labelsOneHot.flat[indexOffset + labelsDense.ravel()] = 1
    return labelsOneHot

In [ ]:
y_train = dense_to_one_hot(trainLabelsFlat,trainLabelsCount)
y_train = y_train.astype(np.uint8)

In [ ]:
testData = data[data.Usage == 'PublicTest']
testPixels = testData.pixels.str.split(" ").tolist()

testPixels = pd.DataFrame(data = testPixels, dtype = int)
testImages = testPixels.values
testImages = testImages.astype(np.float)

In [ ]:
show(testImages[100])

In [ ]:
testLabelsFlat = testData["emotion"].values.ravel()
testLabelsCount = np.unique(testLabelsFlat).shape[0]

y_test = dense_to_one_hot(testLabelsFlat,testLabelsCount)
y_test = y_test.astype(np.uint8)

print(y_test.shape)

In [ ]:
from tensorflow.keras import optimizers

In [ ]:
model = Sequential()

#1. katman
model.add(Conv2D(64, 3, data_format="channels_last", kernel_initializer = "he_normal", input_shape = (48,48,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))

#2. katman
model.add(Conv2D(64,3))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size = (2,2), strides = 2))
model.add(Dropout(0.5))

#3. katman
model.add(Conv2D(64,3))
model.add(BatchNormalization())
model.add(Activation('relu'))

#4. katman
model.add(Conv2D(32,3))
model.add(BatchNormalization())
model.add(Activation('relu'))
#5. katman
model.add(Conv2D(32,3))
model.add(BatchNormalization())
model.add(Activation('relu'))

#6. katman
model.add(Conv2D(32,3))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size = (2,2), strides = 2))
model.add(Dropout(0.6))


#Bağlantı katmanı
model.add(Flatten())
model.add(Dense(128))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

#Çıkış Katmanı
model.add(Dense(7))
model.add(Activation("softmax"))

opt = optimizers.Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name='Nadam')

model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ["accuracy"])
model.summary()

In [ ]:
x_train = trainImages.reshape(-1, 48 , 48 ,1)
x_test = testImages.reshape(-1, 48, 48, 1)

In [ ]:
epochs = 100
batchSize = 32
hist = model.fit(x_train, y_train,
                epochs = epochs, shuffle = True,
                batch_size = batchSize, validation_data = (x_test, y_test), verbose = 2)